In [113]:
# Data Imports
import numpy as np
import pandas as pd


# Math
import math

# Plot imports
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

# Machine Learning Imports
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# For evaluating our ML results
from sklearn import metrics

# Dataset Import
import statsmodels.api as sm

In [101]:
# Reading csv files
# women = pd.read_csv('athletes-women.csv')
# men = pd.read_csv('athletes-men.csv')
# all_ath = pd.read_csv('athletes-all.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

In [102]:
#ensure IDs are int
# women['athlete_id']=women['athlete_id'].astype(int)
# men['athlete_id']=men['athlete_id'].astype(int)
# all_ath['athlete_id']=women['athlete_id'].astype(int)
# #drop nas
# all_ath = all_ath[all_ath['name'].notna()]
# #drop columns 
# all_ath.drop(columns=['team', 'region','affiliate','gender','helen','grace','filthy50','fgonebad','eat','background'])
# #Left join to get all rows in all_ath that are not in women
# df_all = all_ath.merge(women.drop_duplicates(), on=['athlete_id'], how='left', indicator=True)
# df_all.head()

In [114]:
# Reading csv files
women = pd.read_csv('athletes-women.csv')
men = pd.read_csv('athletes-men.csv')
athletes = pd.read_csv('athletes-all.csv')

# athletes who made it into the games
games = pd.concat([women, men])
athletes.head()

/Applications/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,athlete_id,name,region,team,affiliate,gender,age,height,weight,fran,...,deadlift,backsq,pullups,eat,train,background,experience,schedule,howlong,retrieved_datetime
0,2554.0,Pj Ablang,South West,Double Edge,Double Edge CrossFit,Male,24.0,70.0,166.0,NaN,...,400.0,305.0,NaN,NaN,I workout mostly at a CrossFit Affiliate|I hav...,I played youth or high school level sports|I r...,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 2x a week|,4+ years|,NaN
1,3517.0,Derek Abdella,NaN,NaN,NaN,Male,42.0,70.0,190.0,NaN,...,NaN,NaN,NaN,NaN,I have a coach who determines my programming|I...,I played youth or high school level sports|,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 2x a week|,4+ years|,NaN
2,4691.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5164.0,Abo Brandon,Southern California,LAX CrossFit,LAX CrossFit,Male,40.0,67.0,NaN,211.0,...,375.0,325.0,25.0,I eat 1-3 full cheat meals per week|,I workout mostly at a CrossFit Affiliate|I hav...,I played youth or high school level sports|,I began CrossFit by trying it alone (without a...,I usually only do 1 workout a day|,4+ years|,NaN
4,5286.0,Bryce Abbey,NaN,NaN,NaN,Male,32.0,65.0,149.0,206.0,...,NaN,325.0,50.0,I eat quality foods but don't measure the amount|,I workout mostly at a CrossFit Affiliate|I inc...,I played college sports|,I began CrossFit by trying it alone (without a...,I usually only do 1 workout a day|I strictly s...,1-2 years|,NaN


In [115]:
df = athletes.merge(games.drop_duplicates(), on=['athlete_id'], 
                   how='left', indicator=True)

In [116]:
# athletes that did not make it
temp = df.loc[df['_merge'] == 'left_only']
# athletes that made it
temp2 = df.loc[df['_merge'] == 'both']
temp2

,athlete_id,name_x,region_x,team,affiliate,gender,age_x,height_x,weight_x,fran_x,...,candj_y,snatch_y,deadlift_y,backsq_y,pullups_y,coach,certification,multiple,howlong_y,_merge
11,11003.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,305.0,230.0,425.0,425.0,61.0,0.0,1.0,1.0,2.0,both
145,36133.0,Joe Adams,Mid Atlantic,NaN,NaN,Male,31.0,71.0,160.0,245.0,...,285.0,226.0,455.0,345.0,55.0,0.0,0.0,0.0,0.0,both
223,18511.0,Lorenzo Aguilera,NaN,NaN,NaN,Male,28.0,60.0,NaN,NaN,...,287.0,220.0,441.0,397.0,50.0,0.0,1.0,1.0,2.0,both
400,2318.0,Danial Alexander,NaN,NaN,NaN,Male,36.0,70.0,170.0,NaN,...,355.0,293.0,515.0,440.0,60.0,0.0,1.0,1.0,2.0,both
499,83656.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,350.0,285.0,525.0,465.0,60.0,1.0,1.0,1.0,2.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393477,25518.0,Marie Hunter,NaN,NaN,NaN,Female,37.0,NaN,NaN,NaN,...,145.0,125.0,255.0,200.0,50.0,0.0,1.0,1.0,4.0,both
395647,104943.0,Kat Lieb,NaN,NaN,NaN,Female,36.0,66.0,160.0,449.0,...,215.0,140.0,325.0,240.0,60.0,0.0,0.0,1.0,3.0,both
399404,112066.0,Jackie Phillips,NaN,NaN,NaN,Female,27.0,NaN,NaN,NaN,...,110.0,88.0,265.0,187.0,44.0,0.0,1.0,0.0,2.0,both
403054,95973.0,Crystal Taibi,NaN,NaN,NaN,Female,31.0,69.0,170.0,NaN,...,200.0,155.0,325.0,253.0,29.0,0.0,1.0,1.0,3.0,both


In [117]:
# taking only the columns we want
# df_all = df[['athlete_id', 'age_y','age_x','fran_x', 'fran_y', 'run5k_x', 'run5k_y','region_y','region_x', 'pullups_x', 'pullups_y', '_merge']]
df_all = df[['athlete_id', 'age_x', 'height_x', 'weight_x', 'fran_x', 'run5k_x', 'region_x', 'pullups_x', '_merge', 'train', 'experience', 'schedule', 'howlong_x', 'run400_x', 'candj_x', 'snatch_x', 'deadlift_x', 'backsq_x']]

In [93]:
df_all

,athlete_id,age_x,height_x,weight_x,fran_x,run5k_x,region_x,pullups_x,_merge,train,experience,schedule,howlong_x,run400_x,candj_x,snatch_x,deadlift_x,backsq_x
0,2554.0,24.0,70.0,166.0,NaN,NaN,South West,NaN,left_only,I workout mostly at a CrossFit Affiliate|I hav...,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 2x a week|,4+ years|,NaN,220.0,NaN,400.0,305.0
1,3517.0,42.0,70.0,190.0,NaN,NaN,NaN,NaN,left_only,I have a coach who determines my programming|I...,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 2x a week|,4+ years|,NaN,NaN,NaN,NaN,NaN
2,4691.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5164.0,40.0,67.0,NaN,211.0,NaN,Southern California,25.0,left_only,I workout mostly at a CrossFit Affiliate|I hav...,I began CrossFit by trying it alone (without a...,I usually only do 1 workout a day|,4+ years|,NaN,245.0,200.0,375.0,325.0
4,5286.0,32.0,65.0,149.0,206.0,1081.0,NaN,50.0,left_only,I workout mostly at a CrossFit Affiliate|I inc...,I began CrossFit by trying it alone (without a...,I usually only do 1 workout a day|I strictly s...,1-2 years|,NaN,205.0,150.0,NaN,325.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423014,574489.0,36.0,NaN,NaN,NaN,NaN,Latin America,NaN,left_only,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423015,585696.0,27.0,NaN,NaN,NaN,NaN,Australia,NaN,left_only,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423016,608828.0,44.0,NaN,NaN,NaN,NaN,North West,NaN,left_only,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423017,628881.0,20.0,64.0,61.0,NaN,NaN,Europe,NaN,left_only,I workout mostly at a CrossFit Affiliate|,I began CrossFit with a coach (e.g. at an affi...,I usually only do 1 workout a day|I strictly s...,6-12 months|,NaN,77.0,NaN,80.0,143.0


In [118]:
# res is athletes who have the stats filled out
res = df_all.dropna()
res

,athlete_id,age_x,height_x,weight_x,fran_x,run5k_x,region_x,pullups_x,_merge,train,experience,schedule,howlong_x,run400_x,candj_x,snatch_x,deadlift_x,backsq_x
101,2414.0,44.0,69.0,168.0,175.0,1470.0,South East,34.0,left_only,I workout mostly at a CrossFit Affiliate|,I began CrossFit with a coach (e.g. at an affi...,I usually only do 1 workout a day|I typically ...,4+ years|,85.0,245.0,185.0,435.0,405.0
102,2720.0,24.0,68.0,180.0,126.0,1302.0,South Central,80.0,left_only,I workout mostly at a CrossFit Affiliate|I hav...,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 3+ times a wee...,2-4 years|,58.0,315.0,240.0,475.0,405.0
110,6922.0,31.0,67.0,150.0,244.0,1268.0,South West,42.0,left_only,I workout mostly at a CrossFit Affiliate|I inc...,I began CrossFit by trying it alone (without a...,I usually only do 1 workout a day|I do multipl...,2-4 years|,63.0,230.0,175.0,410.0,330.0
115,12244.0,34.0,69.0,185.0,183.0,1188.0,South Central,51.0,left_only,I workout mostly at a CrossFit Affiliate|I hav...,I began CrossFit by trying it alone (without a...,I do multiple workouts in a day 3+ times a wee...,4+ years|,62.0,275.0,215.0,530.0,450.0
116,12563.0,37.0,72.0,210.0,162.0,1320.0,Mid Atlantic,49.0,left_only,I workout mostly at a CrossFit Affiliate|I inc...,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 2x a week|I ty...,4+ years|,66.0,285.0,205.0,525.0,425.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417920,375097.0,22.0,72.0,192.0,148.0,1185.0,Mid Atlantic,61.0,left_only,I workout mostly at a CrossFit Affiliate|I wor...,I began CrossFit by trying it alone (without a...,I do multiple workouts in a day 2x a week|I ty...,1-2 years|,58.0,295.0,235.0,515.0,425.0
418437,470119.0,36.0,65.0,185.0,388.0,1442.0,North Central,13.0,left_only,I workout mostly at a CrossFit Affiliate|,I began CrossFit with a coach (e.g. at an affi...,I usually only do 1 workout a day|,1-2 years|,65.0,215.0,170.0,410.0,365.0
418961,491159.0,43.0,73.0,187.0,690.0,1590.0,South East,15.0,left_only,I workout mostly at a CrossFit Affiliate|I hav...,I began CrossFit with a coach (e.g. at an affi...,I usually only do 1 workout a day|I typically ...,6-12 months|,80.0,155.0,120.0,305.0,205.0
421436,506969.0,36.0,60.0,196.0,495.0,2100.0,North Central,1.0,left_only,I workout mostly at a CrossFit Affiliate|,I began CrossFit with a coach (e.g. at an affi...,I usually only do 1 workout a day|,1-2 years|,100.0,105.0,42.0,235.0,145.0


In [119]:
# ones who made it who have stats filled out
# TODO: need to figure out how to put the stats that we manually found in this df
res.loc[res['_merge'] == 'both']


,athlete_id,age_x,height_x,weight_x,fran_x,run5k_x,region_x,pullups_x,_merge,train,experience,schedule,howlong_x,run400_x,candj_x,snatch_x,deadlift_x,backsq_x
2919,13006.0,36.0,71.0,180.0,183.0,1290.0,Mid Atlantic,40.0,both,I workout mostly at a CrossFit Affiliate|,I began CrossFit by trying it alone (without a...,I do multiple workouts in a day 1x a week|I st...,4+ years|,90.0,255.0,185.0,435.0,335.0
9912,9321.0,31.0,72.0,243.0,173.0,1500.0,Australia,22.0,both,"I workout mostly at home, work, or a tradition...",I began CrossFit by trying it alone (without a...,I do multiple workouts in a day 2x a week|I do...,2-4 years|,75.0,353.0,287.0,573.0,507.0
15609,2901.0,37.0,71.0,198.0,140.0,1260.0,Europe,44.0,both,I workout mostly at a CrossFit Affiliate|I wri...,I began CrossFit by trying it alone (without a...,I do multiple workouts in a day 3+ times a wee...,4+ years|,61.0,298.0,243.0,551.0,441.0
18410,2829.0,23.0,68.0,173.0,237.0,1540.0,South East,45.0,both,I workout mostly at a CrossFit Affiliate|,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 3+ times a wee...,4+ years|,75.0,275.0,225.0,405.0,355.0
19013,5457.0,32.0,64.0,140.0,172.0,1141.0,South West,45.0,both,I workout mostly at a CrossFit Affiliate|I wor...,I began CrossFit by trying it alone (without a...,I do multiple workouts in a day 3+ times a wee...,4+ years|,62.0,265.0,185.0,455.0,405.0
22119,7472.0,30.0,70.0,161.0,225.0,1116.0,Central East,46.0,both,I workout mostly at a CrossFit Affiliate|,I began CrossFit with a coach (e.g. at an affi...,I usually only do 1 workout a day|I typically ...,Less than 6 months|2-4 years|,60.0,245.0,205.0,400.0,325.0
22216,9846.0,29.0,73.0,173.0,211.0,1466.0,Central East,56.0,both,I workout mostly at a CrossFit Affiliate|,I began CrossFit by trying it alone (without a...,I usually only do 1 workout a day|I strictly s...,4+ years|,69.0,285.0,240.0,455.0,380.0
28410,4777.0,27.0,67.0,182.0,172.0,1253.0,South West,68.0,both,I incorporate CrossFit.com workouts|I write my...,I began CrossFit by trying it alone (without a...,I do multiple workouts in a day 2x a week|I ty...,1-2 years|,60.0,295.0,242.0,525.0,400.0
30655,21003.0,34.0,70.0,185.0,143.0,1100.0,North Central,55.0,both,I workout mostly at a CrossFit Affiliate|I wri...,I began CrossFit by trying it alone (without a...,I do multiple workouts in a day 3+ times a wee...,2-4 years|,55.0,285.0,235.0,425.0,385.0
30657,43761.0,24.0,71.0,195.0,145.0,1220.0,Mid Atlantic,76.0,both,I workout mostly at a CrossFit Affiliate|I wri...,I began CrossFit by trying it alone (without a...,I do multiple workouts in a day 3+ times a week|,4+ years|,58.0,335.0,300.0,540.0,480.0


In [120]:
# ones who have not made it who have stats filled out, temp while cleaning up
not_make = res.loc[res['_merge'] == 'left_only']
not_make['howlong_x'] = not_make['howlong_x'].str.slice(0, 6)

#function to return numeric value for howlong
# def howlong_num(s):
#     if np.where(s.str.contains("6-12"), 0, 1)==0:
#         return 1
#     elif np.where(s.str.contains("1-2"), True, False):
#         return 2
#     elif np.where(s.str.contains("2-4"), True, False):
#         return 3
#     elif np.where(s.str.contains("4+"), True, False):
#         return 4
#     else:
#         return 0

col         = 'howlong_x'
conditions  = [ not_make[col].str.contains("Less"), not_make[col].str.contains("6-12"), not_make[col].str.contains("1-2"), not_make[col].str.contains("2-4"), not_make[col].str.contains("4+") ]
choices     = [0,1,2,3,4]
    
not_make["howlong"] = np.select(conditions, choices)

not_make['coach'] = np.where(not_make['train'].str.contains("have a coach"), 1, 0)
not_make['certification'] = np.where(not_make['experience'].str.contains("Level 1"), 1, 0)
not_make['multiple'] = np.where(not_make['schedule'].str.contains("multiple workouts"), 1, 0)




print(not_make.columns)
not_make.head()

Index(['athlete_id', 'age_x', 'height_x', 'weight_x', 'fran_x', 'run5k_x',
       'region_x', 'pullups_x', '_merge', 'train', 'experience', 'schedule',
       'howlong_x', 'run400_x', 'candj_x', 'snatch_x', 'deadlift_x',
       'backsq_x', 'howlong', 'coach', 'certification', 'multiple'],
      dtype='object')


/var/folders/1_/slt2s8nn5cxcjxnzwzm7bn2m0000gn/T/ipykernel_6964/3565838792.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_make['howlong_x'] = not_make['howlong_x'].str.slice(0, 6)
/var/folders/1_/slt2s8nn5cxcjxnzwzm7bn2m0000gn/T/ipykernel_6964/3565838792.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_make["howlong"] = np.select(conditions, choices)
/var/folders/1_/slt2s8nn5cxcjxnzwzm7bn2m0000gn/T/ipykernel_6964/3565838792.py:24: SettingWithCopyWarning: 
A value is trying to be set on a co

,athlete_id,age_x,height_x,weight_x,fran_x,run5k_x,region_x,pullups_x,_merge,train,...,howlong_x,run400_x,candj_x,snatch_x,deadlift_x,backsq_x,howlong,coach,certification,multiple
101,2414.0,44.0,69.0,168.0,175.0,1470.0,South East,34.0,left_only,I workout mostly at a CrossFit Affiliate|,...,4+ yea,85.0,245.0,185.0,435.0,405.0,4,0,0,0
102,2720.0,24.0,68.0,180.0,126.0,1302.0,South Central,80.0,left_only,I workout mostly at a CrossFit Affiliate|I hav...,...,2-4 ye,58.0,315.0,240.0,475.0,405.0,3,1,1,1
110,6922.0,31.0,67.0,150.0,244.0,1268.0,South West,42.0,left_only,I workout mostly at a CrossFit Affiliate|I inc...,...,2-4 ye,63.0,230.0,175.0,410.0,330.0,3,0,1,1
115,12244.0,34.0,69.0,185.0,183.0,1188.0,South Central,51.0,left_only,I workout mostly at a CrossFit Affiliate|I hav...,...,4+ yea,62.0,275.0,215.0,530.0,450.0,4,1,1,1
116,12563.0,37.0,72.0,210.0,162.0,1320.0,Mid Atlantic,49.0,left_only,I workout mostly at a CrossFit Affiliate|I inc...,...,4+ yea,66.0,285.0,205.0,525.0,425.0,4,0,1,1


In [123]:
#not_in_games will contain all athletes that did not make it, with stats, with cleaned up columns
not_in_games = not_make[['athlete_id', 'age_x', 'height_x', 'weight_x', 'fran_x', 'run5k_x', 'region_x', 'pullups_x', 'run400_x', 'candj_x', 'snatch_x', 'deadlift_x', 'backsq_x','howlong','coach','certification','multiple']]
not_in_games.rename(columns={"age_x": "age",'height_x': 'height', 'weight_x':'weight', 'fran_x':'fran', 'run5k_x':'run5k','pullups_x':'pullups', 'run400_x':'run400', 'candj_x':'candj', 'snatch_x':'snatch', 'deadlift_x':'deadlift', 'backsq_x':'backsq'})

not_in_games.head()


,athlete_id,age_x,height_x,weight_x,fran_x,run5k_x,region_x,pullups_x,run400_x,candj_x,snatch_x,deadlift_x,backsq_x,howlong,coach,certification,multiple
101,2414.0,44.0,69.0,168.0,175.0,1470.0,South East,34.0,85.0,245.0,185.0,435.0,405.0,4,0,0,0
102,2720.0,24.0,68.0,180.0,126.0,1302.0,South Central,80.0,58.0,315.0,240.0,475.0,405.0,3,1,1,1
110,6922.0,31.0,67.0,150.0,244.0,1268.0,South West,42.0,63.0,230.0,175.0,410.0,330.0,3,0,1,1
115,12244.0,34.0,69.0,185.0,183.0,1188.0,South Central,51.0,62.0,275.0,215.0,530.0,450.0,4,1,1,1
116,12563.0,37.0,72.0,210.0,162.0,1320.0,Mid Atlantic,49.0,66.0,285.0,205.0,525.0,425.0,4,0,1,1


In [110]:
not_in_games.head()

,athlete_id,age_x,height_x,weight_x,fran_x,run5k_x,region_x,pullups_x,_merge,train,...,howlong_x,run400_x,candj_x,snatch_x,deadlift_x,backsq_x,howlong,coach,certification,multiple
101,2414.0,44.0,69.0,168.0,175.0,1470.0,South East,34.0,left_only,I workout mostly at a CrossFit Affiliate|,...,4+ yea,85.0,245.0,185.0,435.0,405.0,4,0,0,0
102,2720.0,24.0,68.0,180.0,126.0,1302.0,South Central,80.0,left_only,I workout mostly at a CrossFit Affiliate|I hav...,...,2-4 ye,58.0,315.0,240.0,475.0,405.0,3,1,1,1
110,6922.0,31.0,67.0,150.0,244.0,1268.0,South West,42.0,left_only,I workout mostly at a CrossFit Affiliate|I inc...,...,2-4 ye,63.0,230.0,175.0,410.0,330.0,3,0,1,1
115,12244.0,34.0,69.0,185.0,183.0,1188.0,South Central,51.0,left_only,I workout mostly at a CrossFit Affiliate|I hav...,...,4+ yea,62.0,275.0,215.0,530.0,450.0,4,1,1,1
116,12563.0,37.0,72.0,210.0,162.0,1320.0,Mid Atlantic,49.0,left_only,I workout mostly at a CrossFit Affiliate|I inc...,...,4+ yea,66.0,285.0,205.0,525.0,425.0,4,0,1,1


In [111]:
not_in_games.rename(columns = {'age_x':'age', 'height_x':'height', 'weight_x': 'weight', 'fran_x': 'fran',
                              'run5k_x': 'run5k', 'region_x': 'region', 'pullups_x': 'pullups',
                               'run400_x': 'run400', 'candj_x' : 'candj', 'snatch_x': 'snatch', 
                               'deadlift_x': 'deadlift', 'backsq_x': 'backsq'}, inplace = True)

/Applications/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [108]:
not_in_games

,athlete_id,age,height,weight,fran,run5k,region,pullups,_merge,train,experience,schedule,howlong_x,run400,candj,snatch,deadlift,backsq
101,2414.0,44.0,69.0,168.0,175.0,1470.0,South East,34.0,left_only,I workout mostly at a CrossFit Affiliate|,I began CrossFit with a coach (e.g. at an affi...,I usually only do 1 workout a day|I typically ...,4+ years|,85.0,245.0,185.0,435.0,405.0
102,2720.0,24.0,68.0,180.0,126.0,1302.0,South Central,80.0,left_only,I workout mostly at a CrossFit Affiliate|I hav...,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 3+ times a wee...,2-4 years|,58.0,315.0,240.0,475.0,405.0
110,6922.0,31.0,67.0,150.0,244.0,1268.0,South West,42.0,left_only,I workout mostly at a CrossFit Affiliate|I inc...,I began CrossFit by trying it alone (without a...,I usually only do 1 workout a day|I do multipl...,2-4 years|,63.0,230.0,175.0,410.0,330.0
115,12244.0,34.0,69.0,185.0,183.0,1188.0,South Central,51.0,left_only,I workout mostly at a CrossFit Affiliate|I hav...,I began CrossFit by trying it alone (without a...,I do multiple workouts in a day 3+ times a wee...,4+ years|,62.0,275.0,215.0,530.0,450.0
116,12563.0,37.0,72.0,210.0,162.0,1320.0,Mid Atlantic,49.0,left_only,I workout mostly at a CrossFit Affiliate|I inc...,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 2x a week|I ty...,4+ years|,66.0,285.0,205.0,525.0,425.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417920,375097.0,22.0,72.0,192.0,148.0,1185.0,Mid Atlantic,61.0,left_only,I workout mostly at a CrossFit Affiliate|I wor...,I began CrossFit by trying it alone (without a...,I do multiple workouts in a day 2x a week|I ty...,1-2 years|,58.0,295.0,235.0,515.0,425.0
418437,470119.0,36.0,65.0,185.0,388.0,1442.0,North Central,13.0,left_only,I workout mostly at a CrossFit Affiliate|,I began CrossFit with a coach (e.g. at an affi...,I usually only do 1 workout a day|,1-2 years|,65.0,215.0,170.0,410.0,365.0
418961,491159.0,43.0,73.0,187.0,690.0,1590.0,South East,15.0,left_only,I workout mostly at a CrossFit Affiliate|I hav...,I began CrossFit with a coach (e.g. at an affi...,I usually only do 1 workout a day|I typically ...,6-12 months|,80.0,155.0,120.0,305.0,205.0
421436,506969.0,36.0,60.0,196.0,495.0,2100.0,North Central,1.0,left_only,I workout mostly at a CrossFit Affiliate|,I began CrossFit with a coach (e.g. at an affi...,I usually only do 1 workout a day|,1-2 years|,100.0,105.0,42.0,235.0,145.0
